In [8]:
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms
import pandas as pd



train_dataset = torch.load('train_dataset.pth')
test_dataset = torch.load('test_dataset.pth')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.mobilenet_v2(pretrained=True)

# Изменяем классификатор под 3 класса + добавляем Dropout
num_classes = 3
model.classifier = nn.Sequential(
    nn.Dropout(0.3),  # Добавили Dropout для борьбы с переобучением
    nn.Linear(model.last_channel, num_classes)
)

# Переносим модель на GPU/CPU
model = model.to(device)

# Замораживаем веса всех слоев, кроме классификатора
for param in model.features.parameters():
    param.requires_grad = False

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Добавляем lr_scheduler для плавного уменьшения learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# Аугментация для обучения (усилили)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Кроп с изменением масштаба
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.GaussianBlur(kernel_size=(3, 3)),  # Добавили размытие
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Аугментация для валидации (без изменений)
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Создаем DataLoader'ы с shuffle=True для обучения
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Тренировка модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_accuracy = train_correct / len(train_dataset)

    # Валидация
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_accuracy = val_correct / len(test_dataset)

    # Обновляем lr_scheduler
    scheduler.step()

    print(f"Epoch {epoch + 1}: Train Acc: {train_accuracy:.2f}, Val Acc: {val_accuracy:.2f}")



C:\Users\igork\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\igork\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Acc: 0.59, Val Acc: 0.67
Epoch 2: Train Acc: 0.85, Val Acc: 0.70
Epoch 3: Train Acc: 0.88, Val Acc: 0.68
Epoch 4: Train Acc: 0.89, Val Acc: 0.77
Epoch 5: Train Acc: 0.92, Val Acc: 0.82
Epoch 6: Train Acc: 0.91, Val Acc: 0.86
Epoch 7: Train Acc: 0.92, Val Acc: 0.88
Epoch 8: Train Acc: 0.93, Val Acc: 0.91
Epoch 9: Train Acc: 0.92, Val Acc: 0.94
Epoch 10: Train Acc: 0.90, Val Acc: 0.94


In [10]:
def evaluate_on_test(model, test_loader, device):
    """
    Функция для проверки модели на тестовом датасете.
    """
    model.eval()
    test_correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)


            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = test_correct / total
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
    return test_accuracy

In [11]:
test_accuracy = evaluate_on_test(model, test_loader, device)


Test Accuracy: 93.94%
